# Projeto 1 - Ciência dos Dados

Nome: Pedro Barão

Nome: Rafael Paolino

___
Carregando algumas bibliotecas:

In [15]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [16]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\barao\Desktop\Insper\CDados\Projeto1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [17]:
filename = 'PlayStation 5.xlsx'

In [18]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevante
0,"olá querido felipe neto, gostaria de pedir um...",1
1,playstation 5 ou xbox séries x — os dois na mi...,1
2,playstation 5\nxbox series s\ne diversos produ...,1
3,eu quero comprar um playstation 5 e um iphone ...,1
4,"@kgluquinhas2 @banjoboox sem graça, e eu tenho...",1


In [19]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevante
0,‼️ofertasbfn‼️\n\n🎮🎮console playstation 5 sony...,1
1,"#setuplockgamer\n\nfala lock, beleza?\n\n""cons...",0
2,ainda não entendi essa revolta toda com os pre...,0
3,confira minha transmissão do meu playstation 4...,1
4,minha mãe vai me dar o playstation 5 eu nem ac...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

 #### Console de videogames Playstation 5
 #### Foram considerados relevantes tweets que abordavam diretamente o console ou jogos e periféricos compatíveis apenas com ele.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [20]:
import re 

def cleanup(text): # função para limpar textos
    
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [21]:
tweets_total = []

for i in range(0,len(train)):

    tweet=train.iloc[i,0]
    tweet=cleanup(tweet)
    tweet=tweet.split()
    for palavra in tweet:

        tweets_total.append(palavra)

tweets_total_series = pd.Series(tweets_total) # pandas series com textos limpos de cada tweet



In [22]:
relevantes=train.loc[train.Relevante==1,:]
tweets_relevantes = []

for i in range(0,len(relevantes)):

    tweet=relevantes.iloc[i,0]
    tweet=cleanup(tweet)
    tweet=tweet.split()
    for palavra in tweet:
        tweets_relevantes.append(palavra)

tabela_absoluta_tweets_relevantes= pd.Series(tweets_relevantes).value_counts(normalize=False) # tabela com as contagens absolutas das palavras dos tweets relevantes 

In [23]:
irrelevantes=train.loc[train.Relevante==0,:]
tweets_irrelevantes = []

for i in range(0,len(irrelevantes)):

    tweet=irrelevantes.iloc[i,0]
    tweet=cleanup(tweet)
    tweet=tweet.split()
    for palavra in tweet:
        tweets_irrelevantes.append(palavra)

tabela_absoluta_tweets_irrelevantes= pd.Series(tweets_irrelevantes).value_counts(normalize=False) # tabela com as contagens absolutas das palavras dos tweets irrelevantes 

In [24]:
numero_de_tweets = train.loc[:,'Treinamento'].size

numero_de_relevantes=train.loc[train.Relevante==1,'Relevante'].size
numero_de_irrelevantes=train.loc[train.Relevante==0,'Relevante'].size

Prob_relev = numero_de_relevantes/numero_de_tweets # probabilidade de ser relevante
Prob_relevc = numero_de_irrelevantes/numero_de_tweets # probabilidade de não ser relevante



In [35]:
def eh_relevante(tweet,i): # função que recebe a string do tweet e retorna 1 ou 0 para relevância ou irrelevância, respectivamente
    try:
        Prob_tweet_dado_relev = 1
        Prob_tweet_dado_relevc = 1
        numero_palavras_possiveis=tweets_total_series.value_counts().size
        tweet=cleanup(tweet)
        tweet=tweet.split()

        for palavra in tweet: # nesse bloco, é calculada a probalilidade de cada palavra dado que ela é relevante ou irrelevante. Essa probabilidade é, então, multiplicada pela probabilidade da palavra anterior, até que se percorra todas as palavras e se encontre a probabilidade do texto inteiro. Para tal, é utilizado Laplace smoothing para permitir o cálculo de probabilidades de palavras inéditas
            try:
                Prob_tweet_dado_relev *= (tabela_absoluta_tweets_relevantes[palavra]+1)/(numero_palavras_possiveis+tabela_absoluta_tweets_relevantes.size)
                Prob_tweet_dado_relevc *= (tabela_absoluta_tweets_irrelevantes[palavra]+1)/(numero_palavras_possiveis+tabela_absoluta_tweets_irrelevantes.size)         
            except KeyError: # caso a palavra não se encontre na tabela de palavras de tweets relevantes ou na de irrelevantes :
                Prob_tweet_dado_relev *= 1/numero_palavras_possiveis
                Prob_tweet_dado_relevc *= 1/numero_palavras_possiveis
        # Sabendo a probabilidade do texto dado que ele é relevante ou não, é possível utilizar o teorema de Bayes para calcular a probabilidade de um dado texto ser relevante ou não. Note que o termo P(tweet) é omitido dos denominadores, pois, como será feita uma comparação entre P(relev|tweet) e P(relevC|tweet), esse termo será cancelado. Por isso, é desnecessário calculá-lo.
        Prob_relev_dado_tweet= Prob_tweet_dado_relev*Prob_relev 
        Prob_relevc_dado_tweet= Prob_tweet_dado_relevc*Prob_relevc

        if Prob_relev_dado_tweet < Prob_relevc_dado_tweet:
            return 1

        else:
            
            return 0
    except:
        pass




In [45]:
result=[]     
for i in range(0,len(train.iloc[:,0])):
    result.append(eh_relevante(train.iloc[i,0],i))


result_series=pd.Series(result)

train["Algorítimo"]=result_series

acertos=[]
for i in range(0,len(train.iloc[:,0])):
    if train.iloc[i,1]==train.iloc[i,2]:
        acertos.append("Sim")
    else:
        acertos.append("Não")

acertos_series=pd.Series(acertos)

train['Certo']=acertos_series

indice_de_acertos=acertos_series.value_counts()[0]/acertos_series.size

print("Índice de Acerto : {:.2f}%".format(indice_de_acertos*100))

Índice de Acerto : 91.33%


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [44]:
result=[]     
for i in range(0,len(test.iloc[:,0])):
    result.append(eh_relevante(test.iloc[i,0],i))


result_series=pd.Series(result)

test["algorítimo"]=result_series

acertos=[]
for i in range(0,len(test.iloc[:,0])):
    if test.iloc[i,1]==test.iloc[i,2]:
        acertos.append("Sim")
    else:
        acertos.append("Não")

acertos_series=pd.Series(acertos)

test['Certo']=acertos_series

indice_de_acertos=acertos_series.value_counts()[0]/acertos_series.size

print("Índice de Acerto : {:.2f}%".format(indice_de_acertos*100))

Índice de Acerto : 60.00%


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**